# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [38]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def nhl_correlation(): 
    # YOUR CODE HERE
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl_df=nhl_df[['team','W','L','year']].drop(0)
    #print(nhl_df[nhl_df['year']==2018])
    nhl_df=nhl_df[nhl_df['year']==2018]
    mask1=nhl_df['team'].isin(['Atlantic Division','Metropolitan Division','Central Division','Pacific Division'])
    nhl_df=nhl_df[~mask1]
    pattern="""(?P<team>[\w\s]*)"""
    nhl_df['team']=nhl_df['team'].str.extract(pattern)
    nhl_df['W']=nhl_df['W'].astype('int')
    nhl_df['L']=nhl_df['L'].astype('int')
    pattern2="""(?P<team>[\w\s\-\.]+)"""
    cities['NHL']=cities['NHL'].str.extract(pattern2)

    nhl_df['team']=nhl_df['team'].replace({"Boston Bruins":"Boston", 
                                       "Buffalo Sabres":"Buffalo",
                                       "Calgary Flames":"Calgary",
                                       "Columbus Blue Jackets":"Columbus",
                                       "Dallas Stars":"Dallas–Fort Worth",
                                       "Detroit Red Wings":"Detroit",
                                       "Edmonton Oilers":"Edmonton",
                                       "Florida Panthers":"Miami–Fort Lauderdale",
                                       "Los Angeles Kings":"Los Angeles",
                                       "New York Islanders":"New York City",
                                       "New York Rangers":"New York City",
                                       "Ottawa Senators":"Ottawa",
                                       "Philadelphia Flyers":"Philadelphia",
                                       "Phoenix Coyotes":"Phoenix",
                                       "Pittsburgh Penguins":'Pittsburgh',
                                       "San Jose Sharks":"San Francisco Bay Area",
                                       "St":"St. Louis",
                                       "Tampa Bay Lightning":"Tampa Bay Area",
                                       "Toronto Maple Leafs":"Toronto",
                                       "Vancouver Canucks":"Vancouver",
                                       "Vegas Golden Knights":"Las Vegas",
                                       "Washington Capitals":"Washington, D.C.",
                                       "Winnipeg Jets":"Winnipeg",
                                       "Carolina Hurricanes":"Raleigh",
                                       "Chicago Blackhawks":"Chicago",
                                       "Colorado Avalanche":"Denver",
                                       "Montreal Canadiens":"Montreal",
                                       "Nashville Predators":"Nashville",
                                       "New Jersey Devils":"New York City",
                                           "Arizona Coyotes":'Phoenix',
                                           "Minnesota Wild":"Minneapolis–Saint Paul"
                                      })
    nhl_df=nhl_df.rename({'team':'city'},axis=1)
    cities=cities.rename({'Metropolitan area':'city','Population (2016 est.)[8]':'population'},axis=1)                                       
    nhl_df["WL"]=nhl_df['W']/(nhl_df['W']+nhl_df['L'])
    nhl_df=nhl_df.groupby("city").agg({"W":np.sum,"L":np.sum,"WL":np.mean})

    result=pd.merge(nhl_df, cities, how='left', on='city')
    result=result.dropna()
    result['population']=result['population'].astype('int64')
    print(result)
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    for n in result['population']:
        population_by_region.append(n)
    for n in result['WL']:
        win_loss_by_region.append(n)
        
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    CORR, PVAL=stats.pearsonr(population_by_region, win_loss_by_region)
    return CORR

nhl_correlation()

                      city    W    L        WL  population  \
1                   Boston   50   20  0.714286     4794447   
2                  Buffalo   25   45  0.357143     1132804   
3                  Calgary   37   35  0.513889     1392609   
4                  Chicago   33   39  0.458333     9512999   
5                 Columbus   45   30  0.600000     2041520   
6        Dallas–Fort Worth   42   32  0.567568     7233323   
7                   Denver   43   30  0.589041     2853077   
8                  Detroit   30   39  0.434783     4297617   
9                 Edmonton   36   40  0.473684     1321426   
10               Las Vegas   51   24  0.680000     2155664   
11             Los Angeles   45   29  0.608108    13310447   
12   Miami–Fort Lauderdale   44   30  0.594595     6066387   
13  Minneapolis–Saint Paul   45   26  0.633803     3551036   
14                Montreal   29   40  0.420290     4098927   
15               Nashville   53   18  0.746479     1865298   
16      

0.002279772528901799

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [53]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def nba_correlation():
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nba_df=nba_df[['team','W','L','year']]
    nba_df=nba_df[nba_df['year']==2018]
    pattern="""(?P<team>[\w\s]*)"""
    nba_df['team']=nba_df['team'].str.extract(pattern)
    nba_df['team']=nba_df['team'].str.strip()
    nba_df['W']=nba_df['W'].astype('int')
    nba_df['L']=nba_df['L'].astype('int')

    nba_df['team']=nba_df['team'].replace({"Toronto Raptors":"Toronto",
                                           "Boston Celtics":"Boston",
                                           "Philadelphia 76ers":"Philadelphia",
                                           "Cleveland Cavaliers":"Cleveland",
                                           "Indiana Pacers":"Indianapolis",
                                           "Miami Heat":"Miami–Fort Lauderdale",
                                           "Milwaukee Bucks":"Milwaukee",
                                           "Washington Wizards":"Washington, D.C.",
                                           "Detroit Pistons":"Detroit",
                                           "Charlotte Hornets":"Charlotte",
                                           "New York Knicks":"New York City",
                                           "Brooklyn Nets":"New York City",
                                           "Chicago Bulls":"Chicago",
                                           "Orlando Magic":"Orlando",
                                           "Atlanta Hawks":"Atlanta",
                                           "Houston Rockets":"Houston",
                                           "Golden State Warriors":"San Francisco Bay Area",
                                           "Portland Trail Blazers":"Portland",
                                           "Oklahoma City Thunder":"Oklahoma City",
                                           "Utah Jazz":"Salt Lake City",
                                           "New Orleans Pelicans":"New Orleans",
                                           "San Antonio Spurs":"San Antonio",
                                           "Minnesota Timberwolves":"Minneapolis–Saint Paul",
                                           "Denver Nuggets":"Denver",
                                           "Los Angeles Clippers":"Los Angeles",
                                           "Sacramento Kings":"Sacramento",
                                           "Dallas Mavericks":"Dallas–Fort Worth",
                                           "Memphis Grizzlies":"Memphis",
                                           "Phoenix Suns":"Phoenix"                                 
                                      })
    nba_df=nba_df.rename({'team':'city'},axis=1)
    cities=cities.rename({'Metropolitan area':'city','Population (2016 est.)[8]':'population'},axis=1)                                       
    nba_df["WL"]=nba_df['W']/(nba_df['W']+nba_df['L'])
    nba_df=nba_df.groupby("city").agg({"W":np.sum,"L":np.sum,"WL":np.mean})

    result=pd.merge(nba_df, cities, how='left', on='city')
    result=result.dropna()
    result['population']=result['population'].astype('int64')
    print(result)
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    for n in result['population']:
        population_by_region.append(n)
    for n in result['WL']:
        win_loss_by_region.append(n)
        
# pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    CORR, PVAL=stats.pearsonr(population_by_region, win_loss_by_region)
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return CORR
nba_correlation()

                      city   W    L        WL  population  \
0                  Atlanta  24   58  0.292683     5789700   
1                   Boston  55   27  0.670732     4794447   
2                Charlotte  36   46  0.439024     2474314   
3                  Chicago  27   55  0.329268     9512999   
4                Cleveland  50   32  0.609756     2055612   
5        Dallas–Fort Worth  24   58  0.292683     7233323   
6                   Denver  46   36  0.560976     2853077   
7                  Detroit  39   43  0.475610     4297617   
8                  Houston  65   17  0.792683     6772470   
9             Indianapolis  48   34  0.585366     2004230   
10             Los Angeles  42   40  0.512195    13310447   
12                 Memphis  22   60  0.268293     1342842   
13   Miami–Fort Lauderdale  44   38  0.536585     6066387   
14               Milwaukee  44   38  0.536585     1572482   
15  Minneapolis–Saint Paul  47   35  0.573171     3551036   
16             New Orlea

-0.15535519103613454

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [61]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    mlb_df=mlb_df[['team','W','L','year']]
    mlb_df=mlb_df[mlb_df['year']==2018]
    mlb_df['team']=mlb_df['team'].str.strip()
    mlb_df['W']=mlb_df['W'].astype('int')
    mlb_df['L']=mlb_df['L'].astype('int')

    mlb_df['team']=mlb_df['team'].replace({"Boston Red Sox":"Boston",
                                           "New York Yankees":"New York City",
                                           "Tampa Bay Rays":"Tampa Bay Area",
                                           "Toronto Blue Jays":"Toronto",
                                           "Baltimore Orioles":"Baltimore",
                                           "Cleveland Indians":"Cleveland",
                                           "Minnesota Twins":"Minneapolis–Saint Paul",
                                           "Detroit Tigers":"Detroit",
                                           "Chicago White Sox":"Chicago",
                                           "Kansas City Royals":"Kansas City",
                                           "Houston Astros":"Houston",
                                           "Oakland Athletics":"San Francisco Bay Area",
                                           "Seattle Mariners":"Seattle",
                                           "Los Angeles Angels":"Los Angeles",
                                           "Texas Rangers":"Dallas–Fort Worth",
                                           "Atlanta Braves":"Atlanta",
                                           "Washington Nationals":"Washington, D.C.",
                                           "Philadelphia Phillies":"Philadelphia",
                                           "New York Mets":"New York City",
                                           "Miami Marlins":"Miami–Fort Lauderdale",
                                           "Milwaukee Brewers":"Milwaukee",
                                           "Chicago Cubs":"Chicago",
                                           "St. Louis Cardinals":"St. Louis",
                                           "Pittsburgh Pirates":"Pittsburgh",
                                           "Cincinnati Reds":"Cincinnati",
                                           "Los Angeles Dodgers":"Los Angeles",
                                           "Colorado Rockies":"Denver",
                                           "Arizona Diamondbacks":"Phoenix",
                                           "San Francisco Giants":"San Francisco Bay Area",
                                           "San Diego Padres":"San Diego"
                                      })
    mlb_df=mlb_df.rename({'team':'city'},axis=1)
    cities=cities.rename({'Metropolitan area':'city','Population (2016 est.)[8]':'population'},axis=1)                                       
    mlb_df["WL"]=mlb_df['W']/(mlb_df['W']+mlb_df['L'])
    mlb_df=mlb_df.groupby("city").agg({"W":np.sum,"L":np.sum,"WL":np.mean})

    result=pd.merge(mlb_df, cities, how='left', on='city')
    result=result.dropna()
    result['population']=result['population'].astype('int64')
    print(result)
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    for n in result['population']:
        population_by_region.append(n)
    for n in result['WL']:
        win_loss_by_region.append(n)
        
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"
    corr,pval=stats.pearsonr(population_by_region, win_loss_by_region)
    return corr
mlb_correlation()

                      city    W    L        WL  population  \
0                  Atlanta   90   72  0.555556     5789700   
1                Baltimore   47  115  0.290123     2798886   
2                   Boston  108   54  0.666667     4794447   
3                  Chicago  157  168  0.483077     9512999   
4               Cincinnati   67   95  0.413580     2165139   
5                Cleveland   91   71  0.561728     2055612   
6        Dallas–Fort Worth   67   95  0.413580     7233323   
7                   Denver   91   72  0.558282     2853077   
8                  Detroit   64   98  0.395062     4297617   
9                  Houston  103   59  0.635802     6772470   
10             Kansas City   58  104  0.358025     2104509   
11             Los Angeles  172  153  0.529231    13310447   
12   Miami–Fort Lauderdale   63   98  0.391304     6066387   
13               Milwaukee   96   67  0.588957     1572482   
14  Minneapolis–Saint Paul   78   84  0.481481     3551036   
15      

0.13918951993280002

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re



def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df=nfl_df[['team','W','L','year']].drop(0)
    nfl_df=nfl_df[nfl_df['year']==2018]
    #print(nfl_df)
    mask1=nfl_df['team'].isin(['AFC North','AFC South','AFC West','NFC East','NFC North','NFC South','NFC West'])
    nfl_df=nfl_df[~mask1]
    
    pattern="""(?P<team>[\w\s]*)"""
    nfl_df['team']=nfl_df['team'].str.extract(pattern)
    nfl_df['W']=nfl_df['W'].astype('int')
    nfl_df['L']=nfl_df['L'].astype('int')
    nfl_df['team']=nfl_df['team'].replace({"New England Patriots":"Boston",
                                           "Miami Dolphins":"Miami–Fort Lauderdale",
                                           "Buffalo Bills":"Buffalo",
                                           "New York Jets":"New York City",
                                           "Baltimore Ravens":"Baltimore",
                                           "Pittsburgh Steelers":"Pittsburgh",
                                           "Cleveland Browns":"Cleveland",
                                           "Cincinnati Bengals":"Cincinnati",
                                           "Houston Texans":"Houston",
                                           "Indianapolis Colts":"Indianapolis",
                                           "Tennessee Titans":"Nashville",
                                           "Jacksonville Jaguars":"Jacksonville",
                                           "Kansas City Chiefs":"Kansas City",
                                           "Los Angeles Chargers":"Los Angeles",
                                           "Denver Broncos":"Denver",
                                           "Oakland Raiders":"San Francisco Bay Area",
                                           "Dallas Cowboys":"Dallas–Fort Worth",
                                           "Philadelphia Eagles":"Philadelphia",
                                           "Washington Redskins":"Washington, D.C.",
                                           "New York Giants":"New York City",
                                           "Chicago Bears":"Chicago",
                                           "Minnesota Vikings":"Minneapolis–Saint Paul",
                                           "Green Bay Packers":"Green Bay",
                                           "Detroit Lions":"Detroit",
                                           "New Orleans Saints":"New Orleans",
                                           "Carolina Panthers":"Charlotte",
                                           "Atlanta Falcons":"Atlanta",
                                           "Tampa Bay Buccaneers":"Tampa Bay Area",
                                           "Los Angeles Rams":"Los Angeles",
                                           "Seattle Seahawks":"Seattle",
                                           "San Francisco 49ers":"San Francisco Bay Area",
                                           "Arizona Cardinals":"Phoenix"
                                      })
    nfl_df=nfl_df.rename({'team':'city'},axis=1)
    cities=cities.rename({'Metropolitan area':'city','Population (2016 est.)[8]':'population'},axis=1)                                       
    nfl_df["WL"]=nfl_df['W']/(nfl_df['W']+nfl_df['L'])
    nfl_df=nfl_df.groupby("city").agg({"W":np.sum,"L":np.sum,"WL":np.mean})

    result=pd.merge(nfl_df, cities, how='left', on='city')
    #result=result.dropna()
    result['population']=result['population'].astype('int64')
    print(result)
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    for n in result['population']:
        population_by_region.append(n)
    for n in result['WL']:
        win_loss_by_region.append(n)
        

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    CORR, PVAL=stats.pearsonr(population_by_region, win_loss_by_region)
    return CORR

nfl_correlation()

                      city   W   L        WL  population  \
0                  Atlanta   7   9  0.437500     5789700   
1                Baltimore  10   6  0.625000     2798886   
2                   Boston  11   5  0.687500     4794447   
3                  Buffalo   6  10  0.375000     1132804   
4                Charlotte   7   9  0.437500     2474314   
5                  Chicago  12   4  0.750000     9512999   
6               Cincinnati   6  10  0.375000     2165139   
7                Cleveland   7   8  0.466667     2055612   
8        Dallas–Fort Worth  10   6  0.625000     7233323   
9                   Denver   6  10  0.375000     2853077   
10                 Detroit   6  10  0.375000     4297617   
11               Green Bay   6   9  0.400000      318236   
12                 Houston  11   5  0.687500     6772470   
13            Indianapolis  10   6  0.625000     2004230   
14            Jacksonville   5  11  0.312500     1478212   
15             Kansas City  12   4  0.75

0.004922112149349393

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values